In [1]:
import sqlite3
import pandas as pd

### 1. Читаем данные

In [2]:
stat = pd.read_csv("statistic_marvel.csv", index_col=0, encoding="UTF-8")
budget = pd.read_csv("budget_marvel.csv", index_col=0, encoding="UTF-8")

In [3]:
stat.head(5)

,Film_id,Original_Title,Company,Rate,Metascore,Minutes,Release
0,1,Iron Man,Marvel,7.9,79,126,2008
1,2,The Incredible Hulk,Marvel,6.7,61,112,2008
2,3,Iron Man 2,Marvel,7.0,57,124,2010
3,4,Thor,Marvel,7.0,57,115,2011
4,5,Captain America: The First Avenger,Marvel,6.9,66,124,2011


In [4]:
budget.head(5)

,Film_id,Original_Title,Budget,Opening_Weekend_USA,Gross_USA,Gross_Worldwide
0,1,Iron Man,140000000,98618668,318604126,585366247
1,2,The Incredible Hulk,150000000,55414050,134806913,263427551
2,3,Iron Man 2,200000000,128122480,312433331,623933331
3,4,Thor,150000000,65723338,181030624,449326618
4,5,Captain America: The First Avenger,140000000,65058524,176654505,370569774


### 2. Создаем БД

In [5]:
conn = sqlite3.connect('hw.db')
mycursor = conn.cursor()

In [6]:
conn.execute("PRAGMA foreign_keys = ON")

#### 2.1. Создаем таблицы

In [7]:
conn.execute('''CREATE TABLE IF NOT EXISTS statistics (
Film_id INTEGER PRIMARY KEY,
Original_Title TEXT,
Company TEXT,
Rate FLOAT, 
Metascore INTEGER, 
Minutes INTEGER, 
Release INTEGER
)''')

In [8]:
conn.execute('''CREATE TABLE IF NOT EXISTS budget (
Film_id INTEGER PRIMARY KEY,
Original_Title TEXT,
Budget INTEGER,
Opening_Weekend_USA INTEGER,
Gross_USA INTEGER,
Gross_Worldwide INTEGER,
FOREIGN KEY (Film_id) REFERENCES statistics(Film_id)
ON UPDATE CASCADE
ON DELETE CASCADE
)''')

#### 2.2. Подгружаем к ним данные

In [9]:
stat.to_sql('statistics', conn, if_exists='append', index=False)

In [10]:
budget.to_sql('budget', conn, if_exists='append', index=False)

In [11]:
conn.execute("SELECT * FROM statistics").fetchone()

(1, 'Iron Man', 'Marvel', 7.9, 79, 126, 2008)

In [12]:
conn.execute("SELECT * FROM budget").fetchone()

(1, 'Iron Man', 140000000, 98618668, 318604126, 585366247)

#### 2.3. Тестируем команду UPDATE

In [13]:
upd = '''UPDATE statistics SET Rate=10 WHERE Original_Title="Captain America: The First Avenger"'''

mycursor.execute(upd)

conn.execute('''SELECT * FROM statistics WHERE Original_Title="Captain America: The First Avenger"''').fetchall()

[(5, 'Captain America: The First Avenger', 'Marvel', 10.0, 66, 124, 2011)]

#### 2.4. Тестируем команду DELETE

In [15]:
dlt = '''DELETE FROM budget WHERE Gross_USA < 300000000'''

mycursor.execute(dlt)

conn.execute('''SELECT * FROM budget WHERE Gross_USA > 300000000''').fetchmany(5)

[(1, 'Iron Man', 140000000, 98618668, 318604126, 585366247),
 (3, 'Iron Man 2', 200000000, 128122480, 312433331, 623933331),
 (6, 'The Avengers', 220000000, 207438708, 623357910, 1518812988),
 (7, 'Iron Man Three', 200000000, 174144585, 409013994, 1214811252),
 (10, 'Guardians of the Galaxy', 170000000, 94320883, 333176600, 772776600)]

#### 2.5. Тестируем команду CASCADE

In [16]:
dlt_cascade = '''DELETE FROM statistics WHERE Film_id < 30'''

mycursor.execute(dlt_cascade)

conn.execute('''SELECT * FROM statistics''').fetchall()

[(30, 'Green Lantern', 'DC', 5.5, 39, 114, 2011),
 (31, 'The Dark Knight Rises', 'DC', 8.4, 78, 164, 2012),
 (32, 'Man of Steel', 'DC', 7.1, 55, 143, 2013),
 (33, 'Batman v Superman: Dawn of Justice', 'DC', 6.5, 44, 151, 2016),
 (34, 'Suicide Squad\xa0', 'DC', 6.0, 40, 123, 2016),
 (35, 'Wonder Woman', 'DC', 7.4, 76, 141, 2017),
 (36, 'Justice League', 'DC', 6.4, 45, 120, 2017),
 (37, 'Aquaman', 'DC', 7.0, 55, 143, 2018),
 (38, 'Shazam!', 'DC', 7.1, 71, 132, 2019),
 (39, 'Joker', 'DC', 8.7, 59, 122, 2019)]

In [ ]:
conn.execute('''SELECT * FROM budget''').fetchall()

### 3. Комитим изменения и отключаемся от БД

In [ ]:
conn.commit()

In [ ]:
mycursor.close()
conn.close()